In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
### Plot
import matplotlib.pyplot as plt
import seaborn as sns
import re
### Text Preprocessing and Natural Language Processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud

In [3]:
df = pd.read_csv("../attributes.csv")

In [4]:
df

,perfume_id,item_name,brand,theme,note,perfumer,gender,tags,url
0,889072,"挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...",挪威心情,辛辣芳香调,"薄荷,苹果,粉红胡椒,肉桂,葡萄,麝香,零陵香豆,雪松",NaN,NaN,"水果,绿叶,麝香,甜",https://www.nosetime.com/889072-
1,161998,"K 混合魔术 哈亚特 Kemi Blending Magic Hayat, 2014",K 混合魔术,辛辣木质调,"苦橙,薰衣草,藏红花,小豆蔻,琥珀,肉桂,沉香(乌木),雪松,莎草,广藿香",Chris Maurice,NaN,"木质,温暖辛辣,沉香,泥土",https://www.nosetime.com/161998-
2,245968,"梅尔 Mair Acqua Di Lusso, 2024",梅尔,水生花香调,"香柠檬,柠檬,小豆蔻,橙花,栀子花,茉莉,椰子,檀香木,香草",NaN,NaN,"白花,柑橘,甜",https://www.nosetime.com/245968-
3,335908,菲奥雷 In Fiore Kashmir,菲奥雷,芳香木质调,"广藿香,橙花油,檀香木",Julie Elliott,NaN,"木质,广藿香,温暖辛辣,树脂",https://www.nosetime.com/335908-
4,146612,"Khadlaj Perfumes Khadlaj Perfumes Nuha, 2023",Khadlaj Perfumes,NaN,"椰子,苹果,天芥菜,茉莉,玫瑰,香草,檀香木,开司米酮,天芥菜,茉莉,玫瑰,香草,檀香木,开...",NaN,NaN,"香草,脂粉,椰子,木质",https://www.nosetime.com/146612-
...,...,...,...,...,...,...,...,...,...
14251,916718,Alex Perfume Alex Perfume Citrodora,Alex Perfume,NaN,"香柠檬,小豆蔻,柠檬马鞭草,葛缕子,绿叶,杜松,木质香,古云香脂,茶叶,葛缕子,绿叶,杜松,...",NaN,NaN,"柑橘,清新辛辣,木质,芳香植物",https://www.nosetime.com/916718-
14252,440932,"米莎拉 Missala Qmentis, 2023",米莎拉,木质花香调,"粉红胡椒,乳香,醛,榄香脂,香柠檬,雪松,肉桂,茉莉,檀香木,皮革,安息香,劳丹脂,橡木苔,...",NaN,NaN,"东方,木质,皮革,柔软辛辣",https://www.nosetime.com/440932-
14253,650306,"气息香氛 Nose Perfumes Sila Сила, 2019",气息香氛,NaN,"茉莉,沉香(乌木),茶叶,橡木,乳香,胡椒,皮革,朗姆酒,香根草,雪松",NaN,NaN,"木质,清新辛辣,芳香植物,皮革",https://www.nosetime.com/650306-
14254,736748,"拉尼尔 Lanier Folie de Minuit, 1955",拉尼尔,NaN,NaN,NaN,NaN,NaN,https://www.nosetime.com/736748-


In [5]:
print(df['perfumer'].isna().mean() * 100) # 70.7 remove this
print(df['gender'].isna().mean() * 100) # 100 remove this
print(df['tags'].isna().mean() * 100) # 8.46 combine
print(df['theme'].isna().mean() * 100) # 21.2 combine
print(df['note'].isna().mean() * 100) # 5.98 combine
print(df['brand'].isna().mean() * 100) # 0.21 can be removed, bcus the brands are normally in the name of the perfume
print(df['item_name'].isna().mean() * 100) # 0


70.7351290684624
100.0
8.466610549943884
21.198092031425364
5.9834455667789
0.21043771043771042
0.0


In [6]:
cols_to_drop = ['perfumer', 'gender', 'brand', 'url']
df = df.drop(cols_to_drop, axis=1)

In [7]:
desc_cols = ['theme', 'note', 'tags']

# Step 1: Generate detail_desc
df['detail_desc'] = df[desc_cols].fillna('').agg(' '.join, axis=1)

# Step 2: Replace empty or whitespace-only strings with NaN
df['detail_desc'] = df['detail_desc'].replace(r'^\s*$', np.nan, regex=True)

# Step 3: (Optional) Drop rows where detail_desc is NaN
df = df.dropna(subset=['detail_desc'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13495 entries, 0 to 14255
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   perfume_id   13495 non-null  int64 
 1   item_name    13495 non-null  object
 2   theme        11234 non-null  object
 3   note         13403 non-null  object
 4   tags         13049 non-null  object
 5   detail_desc  13495 non-null  object
dtypes: int64(1), object(5)
memory usage: 738.0+ KB


## Chinese Tokenisation and TFIDF

In [9]:
%pip install jieba


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import jieba

def tokenize(text):
    return ' '.join(jieba.cut(text))

df['detail_desc_tokenized'] = df['detail_desc'].apply(tokenize)

tfidf = TfidfVectorizer()
tfidf_scores = tfidf.fit_transform(df['detail_desc_tokenized'])



Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/0r/ynlpb1xx2mz3qlbhfqx5f42m0000gn/T/jieba.cache
Loading model cost 0.594 seconds.
Prefix dict has been built successfully.


In [11]:
tfidf_scores.shape

(13495, 944)

## Cosine Similarity

In [40]:
def cos_sim(vector_a, vector_b):
    # Calculate the dot product of the two vectors
    dot_product = np.dot(vector_a, vector_b)

    # Calculate the Euclidean norm (magnitude) of each vector
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)

    # Calculate cosine similarity using the dot product and vector norms
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [41]:
def pairwise_cos_sim(vectors):
    num_vectors = vectors.shape[0]
    similarity_matrix = np.zeros((num_vectors, num_vectors))

    for i in range(num_vectors):
        for j in range(num_vectors):
            # Calculate cosine similarity for each pair of vectors
            similarity_matrix[i, j] = cos_sim(vectors[i], vectors[j])

    return similarity_matrix

## FAISS

In [44]:
import faiss
import numpy as np
from sklearn.decomposition import TruncatedSVD

# Reduce dimensionality (TF-IDF is sparse and high-dimensional)
svd = TruncatedSVD(n_components=100)
dense_embeddings = svd.fit_transform(tfidf_scores)

# Convert the embeddings to float32
dense_embeddings = dense_embeddings.astype('float32')

# Normalize the vectors to unit length (L2 norm)
faiss.normalize_L2(dense_embeddings)

# Build FAISS index (using inner product or cosine similarity)
index = faiss.IndexFlatIP(dense_embeddings.shape[1])

# Add the normalized embeddings to the index
index.add(dense_embeddings)

# Query for top 5 similar items
_, indices = index.search(dense_embeddings, k=5)

# Output indices of top 5 similar items
print("Indices of top 5 similar items:", indices)



: 

## K Nearest Neighbours K = 5 (including itself)

In [12]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(tfidf_scores)  # tfidf_scores is your sparse matrix

# Find top 5 similar perfumes for each
distances, indices = model.kneighbors(tfidf_scores, n_neighbors=5)

In [18]:
import pandas as pd

# Create a DataFrame to store results
perfume_names = df['item_name'].values  # Extract perfume names

# Create an empty list to store the results
similar_perfumes_list = []

# Iterate through each perfume and its top 5 similar perfumes
for i, perfume in enumerate(perfume_names):
    for j, idx in enumerate(indices[i]):
        similar_perfumes_list.append({
            "Perfume": perfume,
            "Similar Perfume": perfume_names[idx],
            "Cosine Distance": distances[i][j]
        })

# Convert the list to a DataFrame
similar_perfumes = pd.DataFrame(similar_perfumes_list)

# Display the DataFrame with recommendations
similar_perfumes


,Perfume,Similar Perfume,Cosine Distance
0,"挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...","挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...",0.000000
1,"挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...","莱卡门 Les Copains Le Bleu, 2000",0.490791
2,"挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...",安东尼奥·班德拉斯 黄金秘密 Antonio Banderas The Golden Sec...,0.510306
3,"挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...",Frankie Garage Frankie Garage Sporty Fragranc...,0.515445
4,"挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...","奥特瑞之香 Autre Parfum Avant-Garde Nuit, 2013",0.523321
...,...,...,...
67470,"Medeau 起源 Medeau Origin, 2021","Medeau 起源 Medeau Origin, 2021",0.000000
67471,"Medeau 起源 Medeau Origin, 2021","Federico Fumo Federico Fumo Luci A Positano, ...",0.444558
67472,"Medeau 起源 Medeau Origin, 2021","Max Philip Max Philip Cyan, 2024",0.541295
67473,"Medeau 起源 Medeau Origin, 2021",Fleur de Florence Fleur de Florence Pure Rose...,0.544079


In [19]:
# Remove self-matching results if present
similar_perfumes = similar_perfumes[similar_perfumes['Perfume'] != similar_perfumes['Similar Perfume']]

In [24]:
similar_perfumes

,Perfume,Similar Perfume,Cosine Distance
1,"挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...","莱卡门 Les Copains Le Bleu, 2000",0.490791
2,"挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...",安东尼奥·班德拉斯 黄金秘密 Antonio Banderas The Golden Sec...,0.510306
3,"挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...",Frankie Garage Frankie Garage Sporty Fragranc...,0.515445
4,"挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...","奥特瑞之香 Autre Parfum Avant-Garde Nuit, 2013",0.523321
6,"K 混合魔术 哈亚特 Kemi Blending Magic Hayat, 2014","Made Vesuna, 2023",0.264867
...,...,...,...
67469,"气息香氛 Nose Perfumes Sila Сила, 2019",DANHERA DANHERA Abanthe,0.384683
67471,"Medeau 起源 Medeau Origin, 2021","Federico Fumo Federico Fumo Luci A Positano, ...",0.444558
67472,"Medeau 起源 Medeau Origin, 2021","Max Philip Max Philip Cyan, 2024",0.541295
67473,"Medeau 起源 Medeau Origin, 2021",Fleur de Florence Fleur de Florence Pure Rose...,0.544079


### Finding Recommendation

In [29]:
recommendations = similar_perfumes[similar_perfumes['Perfume'] == '气息香氛 Nose Perfumes Sila Сила, 2019']['Similar Perfume'].to_list()
recommendations

['劳伦马佐 硬质皮革 Laurent Mazzone Parfums Hard Leather, 2014',
 '奥菲斯之家 House of Orpheus Djinn, 2014',
 'Ghalati  Ghalati Leather Oud, 2023',
 'DANHERA  DANHERA Abanthe']

### K Means Clustering

In [21]:
from sklearn.cluster import KMeans

# Choose the number of clusters (for example, 10)
kmeans = KMeans(n_clusters=10, random_state=42)

# Fit KMeans to the TF-IDF matrix
kmeans.fit(tfidf_scores)

# Add cluster labels to the DataFrame
df['Cluster'] = kmeans.labels_

/Users/isabel/.pyenv/versions/3.11.9/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


                                           item_name  Cluster
0  挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...        4
1         K 混合魔术 哈亚特 Kemi Blending Magic Hayat, 2014        8
2                       梅尔 Mair Acqua Di Lusso, 2024        1
3                               菲奥雷 In Fiore Kashmir        2
4      Khadlaj Perfumes  Khadlaj Perfumes Nuha, 2023        9


In [23]:
df

,perfume_id,item_name,theme,note,tags,detail_desc,detail_desc_tokenized,Cluster
0,889072,"挪威心情 Moods Of Norway Disco Sunset Pour Homme, ...",辛辣芳香调,"薄荷,苹果,粉红胡椒,肉桂,葡萄,麝香,零陵香豆,雪松","水果,绿叶,麝香,甜","辛辣芳香调 薄荷,苹果,粉红胡椒,肉桂,葡萄,麝香,零陵香豆,雪松 水果,绿叶,麝香,甜","辛辣 芳香 调 薄荷 , 苹果 , 粉红 胡椒 , 肉桂 , 葡萄 , 麝香 , 零陵 ...",4
1,161998,"K 混合魔术 哈亚特 Kemi Blending Magic Hayat, 2014",辛辣木质调,"苦橙,薰衣草,藏红花,小豆蔻,琥珀,肉桂,沉香(乌木),雪松,莎草,广藿香","木质,温暖辛辣,沉香,泥土","辛辣木质调 苦橙,薰衣草,藏红花,小豆蔻,琥珀,肉桂,沉香(乌木),雪松,莎草,广藿香 木质...","辛辣 木质 调 苦 橙 , 薰衣草 , 藏红花 , 小 豆蔻 , 琥珀 , 肉桂 , 沉...",8
2,245968,"梅尔 Mair Acqua Di Lusso, 2024",水生花香调,"香柠檬,柠檬,小豆蔻,橙花,栀子花,茉莉,椰子,檀香木,香草","白花,柑橘,甜","水生花香调 香柠檬,柠檬,小豆蔻,橙花,栀子花,茉莉,椰子,檀香木,香草 白花,柑橘,甜","水生 花香调 香 柠檬 , 柠檬 , 小 豆蔻 , 橙花 , 栀子花 , 茉莉 , 椰子...",1
3,335908,菲奥雷 In Fiore Kashmir,芳香木质调,"广藿香,橙花油,檀香木","木质,广藿香,温暖辛辣,树脂","芳香木质调 广藿香,橙花油,檀香木 木质,广藿香,温暖辛辣,树脂","芳香 木质 调 广藿香 , 橙花油 , 檀香木 木质 , 广藿香 , 温暖 辛辣 , 树脂",2
4,146612,"Khadlaj Perfumes Khadlaj Perfumes Nuha, 2023",NaN,"椰子,苹果,天芥菜,茉莉,玫瑰,香草,檀香木,开司米酮,天芥菜,茉莉,玫瑰,香草,檀香木,开...","香草,脂粉,椰子,木质","椰子,苹果,天芥菜,茉莉,玫瑰,香草,檀香木,开司米酮,天芥菜,茉莉,玫瑰,香草,檀香木,...","椰子 , 苹果 , 天 芥菜 , 茉莉 , 玫瑰 , 香草 , 檀香木 , 开司米 酮 ...",9
...,...,...,...,...,...,...,...,...
14250,271665,"科黛 占有 Corday Possession, 1937",醛香花香调,"醛,鼠尾草,白松香,可可果,公丁香,香柠檬,鸢尾花,依兰,天芥菜,紫罗兰,玫瑰,茉莉,康乃馨...","脂粉,鸢尾,香草,花香","醛香花香调 醛,鼠尾草,白松香,可可果,公丁香,香柠檬,鸢尾花,依兰,天芥菜,紫罗兰,玫瑰,...","醛 香 花香调 醛 , 鼠尾草 , 白 松香 , 可可 果 , 公 丁香 , 香 柠檬 ...",9
14251,916718,Alex Perfume Alex Perfume Citrodora,NaN,"香柠檬,小豆蔻,柠檬马鞭草,葛缕子,绿叶,杜松,木质香,古云香脂,茶叶,葛缕子,绿叶,杜松,...","柑橘,清新辛辣,木质,芳香植物","香柠檬,小豆蔻,柠檬马鞭草,葛缕子,绿叶,杜松,木质香,古云香脂,茶叶,葛缕子,绿叶,杜松...","香 柠檬 , 小 豆蔻 , 柠檬 马鞭草 , 葛缕子 , 绿叶 , 杜松 , 木质 香 ...",4
14252,440932,"米莎拉 Missala Qmentis, 2023",木质花香调,"粉红胡椒,乳香,醛,榄香脂,香柠檬,雪松,肉桂,茉莉,檀香木,皮革,安息香,劳丹脂,橡木苔,...","东方,木质,皮革,柔软辛辣","木质花香调 粉红胡椒,乳香,醛,榄香脂,香柠檬,雪松,肉桂,茉莉,檀香木,皮革,安息香,劳丹...","木质 花香调 粉红 胡椒 , 乳香 , 醛 , 榄 香脂 , 香 柠檬 , 雪松 , 肉...",7
14253,650306,"气息香氛 Nose Perfumes Sila Сила, 2019",NaN,"茉莉,沉香(乌木),茶叶,橡木,乳香,胡椒,皮革,朗姆酒,香根草,雪松","木质,清新辛辣,芳香植物,皮革","茉莉,沉香(乌木),茶叶,橡木,乳香,胡椒,皮革,朗姆酒,香根草,雪松 木质,清新辛辣,芳...","茉莉 , 沉香 ( 乌木 ) , 茶叶 , 橡木 , 乳香 , 胡椒 , 皮革 , 朗姆...",7
